In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
pip install git+https://github.com/davidbau/baulab

# One-dimensional GAN math proof demo

Why do GAN's work?

In his [original GAN paper](https://arxiv.org/pdf/1406.2661.pdf), Goodfellow sets up a two-player training game where a generator $G$ is trained to fool a discriminator $D$ which is adversarially trained to distinguish $G$ from a true data distribution (which he calls $data$, and we'll call $T$).

He argues that the optimal solution to $G$ is to perfectly imitate $T$.

This notebook outlines his mathematical argument visually, using a one-dimensional Gaussian model, to help you build intuition about how the adversarial GAN game works.  In this notebook, the true distribtuion $T$ is a one-dimensional Gaussian distribution, and the model generator $G$ is also a Gaussian, parameterized by its mean and variance.

In the cell below, we write code to explicitly plot the following four things:
  * `true_model` is the target distribution $T$
  * `gen_model` is the generative model distribution $G$.
  * `discriminator` is the optimal discriminator $D = \frac{P_T}{P_T + P_G}$ which can be plotted explicitly.
  * `value_fn` is the (negative) cross-entropy loss $V = E_T[\log(D)] + E_G[\log(1-D)]$.
  

In [ ]:
from baulab import PlotWidget, Range, Numberbox, show
import torch, math

def gaussian_model(x, mean, variance):
    return torch.exp(-(x - mean) ** 2 / variance) / torch.sqrt(2 * torch.pi * variance)

def make_gaussian_model(mean, variance):
    return lambda x: gaussian_model(x, torch.tensor(mean), torch.tensor(variance))

def discriminator(x, p_true, p_gen):
    return p_true(x) / (p_true(x) + p_gen(x))

def value_fn(x, p_true, p_gen):
    d = discriminator(x, p_true, p_gen)
    return p_true(x) * torch.log(d) + p_gen(x) * torch.log(1 - d)

loghalf = math.log(0.5)

def value_fn(x, p_true, p_gen, rule='v'):
    d = discriminator(x, p_true, p_gen)
    constant = loghalf if rule == 'js' else 0
    return p_true(x) * (torch.log(d) - constant) + p_gen(x) * (torch.log((1 - d).clamp(1e-50)) - constant)

def js_redraw_rule(fig, gen_mean=0.0, gen_variance=1.0, true_mean=3.0, true_variance=0.5, rule='v'):
    if len(fig.axes) == 1:
       ax2 = fig.axes[0].twinx()
    [ax, ax2] = fig.axes
    x_range = torch.arange(-3, 6, 0.1, dtype=torch.double)
    true_model = make_gaussian_model(true_mean, true_variance)
    gen_model = make_gaussian_model(gen_mean, gen_variance)
    ax2.clear()
    ax2.set_ylim(-2.05, 2.05)
    ax2.set_ylabel('$JS(G, T)$ divergence' if rule == 'js' else 'cross-entropy $V$')
    constant = '-\log{0.5}' if rule == 'js' else ''
    ax2.fill_between(x_range, value_fn(x_range, true_model, gen_model, rule=rule),
                     alpha=0.5, color='red', lw=0,
                     label=f'${rule.upper()} = E_T[\log(D){constant}] + E_G[\log(1-D){constant}]$')
    ax.clear()
    ax.set_title(('JS divergence: minimize JSD' if rule == 'js' else 'The GAN game: minimize $V$') + ' by fooling $D$')
    ax.plot(x_range, gen_model(x_range), linewidth=3, label='$P_G$ generator distribution')
    ax.plot(x_range, true_model(x_range), label='$P_T$ target distribution')
    ax.plot(x_range, discriminator(x_range, true_model, gen_model), label='$D$ discriminator classifies T')
    ax.set_ylabel('probability density $P_G$, $P_T$, probability $D$')
    ax.set_ylim(-0.05, 1.05)
    ax.legend(*(a + b for a, b in zip(*[axN.get_legend_handles_labels() for axN in [ax, ax2]])),
               loc='upper left')

# Plotting the concepts

Let's start with a generator $G$ that is wrong: it is just centered at zero, and far away from the true distribution $T$ centered somewhere else.

How wrong is it?  Can get information about that by training a classifier $D: x \rightarrow y$ that aims to distingush between true $x \sim T$ drawn from the true distribution and generated $x \sim G$ sampled from our generative model.  We want $D(x)$ to output a number between 0 and 1 that says how likely we should think $x$ is to be a true sample, so we want it to be near-one when $x$ is close to the $T$ mean and near-zero when $x$ is close to the $G$ mean.

Here is a picture of $G$ in blue, $T$ is orange, and the optimal discriminator $D$ in green.

Notice that $D$ is near-zero where $G$ is dense, and near-one where $T$ is dense.

What is this $V$ business?

In [ ]:
PlotWidget(js_redraw_rule)

## Training D by maximizing (the negative cross entropy) V

The area over the red curve $V$ above is plotting the negation of the cross-entropy term that is normally estimated (integrated by taking random samples) when we classify $D$.  Because the log terms in cross-entropy can be thought of bits of surprise $D$ would charge itself for being wrong about a given $x$, we can think of the whole integral as a measure of the total amount of suprise $D$ will see over the whole distribution. It's a negative number that the optimal $D$ is trying to make as big (near zero) as possible; a perfect classifier would be perfectly certain and always zero.

As an integral, the cross-entropy loss that we optimize when we train $D$ is this:

$$ V = E_T[\log(D(x))] + E_G[\log(1-D(x))] $$
$$ = \int P_T(x) \log(D(x)) + P_G(x) \log(1-D(x)) dx $$

That is just a fancy way of saying, for any $x$, we want $D(x) = y$ where $y$ maximizes the inside of the integral at every $x$:

$$ P_T(x) \log(y) + P_G(x) \log(1-y) $$

Using calculus, taking the derivative and setting it to zero, we can see that is just a fancy way to say something even simpler:

$$ 0 = \frac{d}{dy} ( P_T(x) \log(y) + P_G(x) \log(1-y) ) = \frac{P_T(X)}{y} - \frac{P_G(X)}{1- y} $$
$$ D(x) = y = \frac{P_T(x)}{P_T(x) + P_G(x)} $$

That is, the optimal $D$ just tells the ratio between the probabilitiy that $x$ came from the true distribution, divided by the total probabilitiy that it might have come from the sum of both distributions.  Simple!

The reason we normally need to train models of $D$ instead of just dividing the $P$ probabilities, is that in the real world, we do not have explicit density functions $P_T$ and $P_G$.  We do not know the probabilities. But in this toy example, we do lived in a world where we can calculate explicit density functions for our distributions, so we can skip the neural network training for $D$ and just calculate $D(x)$ explicitly by just calculating this fraction.

## Training G by making D as bad as possible

Our real goal is to train $G$ into a good model of the true distribution $T$. At the start it is a miserably bad model, because given any point $x$, it is very easy to tell the difference between whether $x$ was sampled from $G$ or sampled from $T$.  The discriminator $D$ is almost always very sure of itself, and the negative cross-entropy value $V$ is always very big (near zero).

But if we move $G$ around, we can make the job harder for $D$.

Goodfellow proves that the same cross-entropy $V$ that is used to train $D$ is also a good global estimate of how good the generator $G$ is.  The lower $V$ is, the harder it is for $D$ to tell between $G$ and $T$, and the better $G$ is.

The very best $G$ is the one that achieves the minimum $V$.  In other words, the best generator is the winner of a *minimax* game, *minimizing* the *maximum* $V$ that can be obtained by a discriminator that can train on samples of $G$.

Use the sliders below to play the minimax game as $G$.  See how making $G$ closer to $T$ makes $D$ more ambiguous.  This also makes the negative cross-entropy $V$ smaller (more negative) as it's more likely to be uncertain.

In [ ]:
plot = PlotWidget(js_redraw_rule, figsize=(8, 5), dpi=100)
mean_slider = Range(value=plot.prop('gen_mean'), min=-5, max=5, step=0.01)
mean_input = Numberbox(value=plot.prop('gen_mean'))
var_slider = Range(value=plot.prop('gen_variance'), min=0.1, max=1.9, step=0.01)
var_input = Numberbox(value=plot.prop('gen_variance'))
show([plot,
      [show.style(textAlign='right'), 'Generator Mean', show.style(flex=2), mean_slider, mean_input],
      [show.style(textAlign='right'), 'Generator Variance', show.style(flex=2), var_slider, var_input]])

## Proof sketch

With this intuition in hand, it is worth reading the Goodfellow paper proof to see how the argument goes.  Basically, he goes through two steps:

  1. First he notices that when $G = T$, we have the constant and perfectly confused discriminator $D(x) = \frac{1}{2}$ exactly, and this gives us $V = -\log 4$.
  2. Then by rearranging terms, he observes that, in the general case where $G$ differs from $T$, you have $V = -\log 4 + 2 \cdot \text{JS} (G, T)$ where $\text{JS}$ is the Jensen-Shannon divergence between the two distributions.
  
JS divergence averages the KL divergence between $G$ and $T$ and the 50-50 mix distribution between $G$ and $T$, that is: $\text{JS}(G, T) = \text{KL}(G; M)/2 + \text{KL}(T; M)/2$ where $M=(G+T)/2$. Like KL divergence, JS divergence is never negative, so $V$ is never smaller than $-\log 4$, and the distribution $G=T$ is a global optimum.  Also when JS divergence is zero, it means that $G=T$.

Furthermore, so we can see that the objective of minimizing $V$ comes down to reducing $\text{JS}(G, T)$. 

This theoretical optimum result is important, because it means, if we can figure out how to actually solve the optimization perfectly in practice, we would be able to perfectly imitate the target distribution.

You can see how JS divergence is just a constant offset from $V$, and always positive, in the pair of plots below.

In [ ]:
plot = PlotWidget(js_redraw_rule, figsize=(6, 5))
plot_js = PlotWidget(js_redraw_rule, rule='js', figsize=(6, 5),
                     gen_mean=plot.prop('gen_mean'), gen_variance=plot.prop('gen_variance'))
mean_slider = Range(value=plot.prop('gen_mean'), min=-5, max=5, step=0.01)
mean_input = Numberbox(value=plot.prop('gen_mean'))
var_slider = Range(value=plot.prop('gen_variance'), min=0.1, max=1.9, step=0.01)
var_input = Numberbox(value=plot.prop('gen_variance'))
show([[plot, show.style(margin=0), plot_js],
      [show.style(textAlign='right'), 'Generator Mean', show.style(flex=2), mean_slider, mean_input],
      [show.style(textAlign='right'), 'Generator Variance', show.style(flex=2), var_slider, var_input]])

## Intuition about limitations

You can get some intuition about the weaknesses of GAN training from the demo.  Think about:

  1. Where does $V$ change very quickly, and where does it stay mostly unchanged?
  2. What does this tell you about what will be hard for an optimizer to do, when training $G$ to optimize this $V$?

Go back and play with the demo while thinking about these questions.

## Why is adversarial training interesting?

This demo just demonstrates why the GAN loss works, mathematically.

But the real reason GANs are interesting is because of their computational form.  There are three benefits.

  1. It is very familiar, just like training a classifier.  We are very accustomed to training a classifier $D$ by optimizing estimates of cross-entropy loss $V$ taken from random samples of data.  GAN training just says that you can <em>also</em> train a generator $G$ by optimizing the same $V$, but <em>adversarially</em>, minimizing the maximum solution, making it as hard as possible to classify $G$ versus $T$.
  
  2. It gives us a smart perceptual loss function $D$.  Because $D$ is part of the loss function for $G$, it means that we can train a very smart $D$ in the hope of teaching a very smart $G$.  In practice, the big advantage is that we can train a $D$ that knows how to do computer vision, looking for edges and textures.  When we train $G$ to fool $D$, it must learn to draw edges and textures, the same types of visual features humans perceive.
  
  3. It gives us a sampling model $G$.  GAN training is set up is to train a $G$ that gives you random samples of data, all in one shot.  That is different from previous approaches that learn a density function, where once you're done, you need to figure out how to sample from the function.  With GANs, once training is done, we can generate new instances by just running $G$ once.  No new sampling problem to solve.